In [18]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

## Index Migration (v1 to v2)

This notebook is used to maintain data model parity with older indexes for version 2.0 of GraphRAG. If you have a pre-2.0 index and need to migrate without re-running the entire pipeline, you can use this notebook to only update the pieces necessary for alignment. If you have a pre-1.0 index, please run the v1 migration notebook first!

NOTE: we recommend regenerating your settings.yml with the latest version of GraphRAG using `graphrag init`. Copy your LLM settings into it before running this notebook. This ensures your config is aligned with the latest version for the migration. This also ensures that you have default vector store config, which is now required or indexing will fail.

WARNING: This will overwrite your parquet files, you may want to make a backup!

In [ ]:
# This is the directory that has your settings.yaml
PROJECT_DIRECTORY = "<your project directory>"

In [20]:
from pathlib import Path

from graphrag.config.load_config import load_config
from graphrag.storage.factory import StorageFactory

config = load_config(Path(PROJECT_DIRECTORY))
storage_config = config.output.model_dump()
storage = StorageFactory().create_storage(
    storage_type=storage_config["type"],
    kwargs=storage_config,
)

In [21]:
def remove_columns(df, columns):
    """Remove columns from a DataFrame, suppressing errors."""
    df.drop(labels=columns, axis=1, errors="ignore", inplace=True)

In [ ]:
import numpy as np
from graphrag_storage.tables.parquet_table_provider import ParquetTableProvider

# Create table provider from storage
table_provider = ParquetTableProvider(storage)

final_documents = await table_provider.read_dataframe("create_final_documents")
final_text_units = await table_provider.read_dataframe("create_final_text_units")
final_entities = await table_provider.read_dataframe("create_final_entities")
final_covariates = await table_provider.read_dataframe("create_final_covariates")
final_nodes = await table_provider.read_dataframe("create_final_nodes")
final_relationships = await table_provider.read_dataframe("create_final_relationships")
final_communities = await table_provider.read_dataframe("create_final_communities")
final_community_reports = await table_provider.read_dataframe(
    "create_final_community_reports"
)

# we've renamed document attributes as metadata
if "attributes" in final_documents.columns:
    final_documents.rename(columns={"attributes": "metadata"}, inplace=True)

# we're removing the nodes table, so we need to copy the graph columns into entities
graph_props = (
    final_nodes.loc[:, ["id", "degree", "x", "y"]].groupby("id").first().reset_index()
)
final_entities = final_entities.merge(graph_props, on="id", how="left")
# we're also persisting the frequency column
final_entities["frequency"] = final_entities["text_unit_ids"].count()


# we added children to communities to eliminate query-time reconstruction
parent_grouped = final_communities.groupby("parent").agg(
    children=("community", "unique")
)
final_communities = final_communities.merge(
    parent_grouped,
    left_on="community",
    right_on="parent",
    how="left",
)
# replace NaN children with empty list
final_communities["children"] = final_communities["children"].apply(
    lambda x: x if isinstance(x, np.ndarray) else []  # type: ignore
)

# add children to the reports as well
final_community_reports = final_community_reports.merge(
    parent_grouped,
    left_on="community",
    right_on="parent",
    how="left",
)

# we renamed all the output files for better clarity now that we don't have workflow naming constraints from DataShaper
await table_provider.write_dataframe("documents", final_documents)
await table_provider.write_dataframe("text_units", final_text_units)
await table_provider.write_dataframe("entities", final_entities)
await table_provider.write_dataframe("relationships", final_relationships)
await table_provider.write_dataframe("covariates", final_covariates)
await table_provider.write_dataframe("communities", final_communities)
await table_provider.write_dataframe("community_reports", final_community_reports)

# delete all the old versions
await storage.delete("create_final_documents.parquet")
await storage.delete("create_final_text_units.parquet")
await storage.delete("create_final_entities.parquet")
await storage.delete("create_final_nodes.parquet")
await storage.delete("create_final_relationships.parquet")
await storage.delete("create_final_covariates.parquet")
await storage.delete("create_final_communities.parquet")
await storage.delete("create_final_community_reports.parquet")